In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
from matplotlib import pyplot as plt
from matplotlib.image import imread
import cv2
import os
import numpy as np
from data_preprocessor import TransformDataset
from functools import reduce
from cokenet_model import CokeModel
from tensorflow.keras.utils import to_categorical
from rectified_adam import RAdam

In [3]:
def get_mapped_list(path):
    return [str(file) for r, _, f in os.walk(path) for file in f if any(map(str(file).lower().__contains__, ['.png', '.jpg', '.jpeg']))]  

def resize_image(img_path, output_shape):
    im = imread(img_path)
    resized = cv2.resize(im, (output_shape, output_shape), interpolation=cv2.INTER_AREA)
    return cv2.normalize(resized, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, 
                                 dtype = cv2.CV_32F).astype(np.uint8)

def resize_directory(base_path, file_lst, output_shape=160):
    for im_file in file_lst:
        rel_path = os.path.join(base_path, im_file)
        try:
            if imread(rel_path).shape == (output_shape, output_shape, 3):
                continue
            resized_im = resize_image(rel_path, output_shape)
            resized_im[resized_im<0] = 0
            plt.imsave(os.path.join(base_path, im_file), resized_im)
            if imread(os.path.join(base_path, im_file)).shape == (output_shape, output_shape, 4):
                resized_im = imread(rel_path)[:, :, :3]
                resized_im[resized_im<0] = 0
                plt.imsave(os.path.join(base_path, im_file), resized_im)
        except:
            os.remove(rel_path)
    return

def create_augmentation_set(original_path, transformations, severity = list(map(lambda x: x+1, np.arange(5)))):
    output_path = original_path + '_augmentation'
    file_lst = get_mapped_list(original_path)
    func = TransformDataset()
    has_severity = ['rotate_np', 'flip_rotate', 'perform_swirl_transformation', 'perform_random_affine_transform', 
                    'add_multiplicative_noise', 'add_shot_noise', 'add_gaussian_noise', 'add_impulse_noise', 
                    'add_glass_blur', 'add_gaussian_blur']
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    for im_file in file_lst:
        im = imread(os.path.join(original_path, im_file))
        file_name, extension = im_file.split('.')
        
        for transF in transformations:
            if transF not in has_severity:
                try:
                    plt.imsave(os.path.join(output_path, ".".join(["_".join([file_name, transF]), extension])), 
                               func.return_function(transF, im))
                except:
                    print ("Failed to augment file = {}".format(file_name))
                    continue
            else:
                for severity_ in severity:
                    try:
                        plt.imsave(os.path.join(output_path, ".".join(["_".join([file_name, transF]), extension])), 
                           func.return_function(transF, im, severity_))
                    except:
                        print ("Failed to augment file = {}".format(file_name))
                        continue
        #print ("Successfully created Augmentation Set for {}".format(im_file))
    return output_path

In [4]:
def process_prepare_directories(dir_, transformations, label, create_augmentation=True, 
                                data_map=[], paths_map=[], label_map=[]):
    dataset = get_mapped_list(dir_)
    resize_directory(dir_, dataset)
    
    data_map.append(dataset)
    paths_map.append(dir_)
    label_map.append(label)
    
    if create_augmentation:
        dir_augmented = create_augmentation_set(dir_, transformations)
        dataset_augmented = get_mapped_list(dir_augmented)
        data_map.append(dataset_augmented)
        paths_map.append(dir_augmented)
        label_map.append(label)
        
    return data_map, paths_map, label_map

def create_dataset(data_map, path_map, label_map, output_shape):
    h, w, c = output_shape, output_shape, 3
    size_data = reduce(lambda x,y: x+y, map(lambda x: len(x), data_map))
    X = np.zeros((size_data, h, w, c), dtype=np.float32)
    Y = np.zeros((size_data))
    x_ptr = 0
    for idx in range(len(data_map)):
        if label_map[idx]:
            Y[x_ptr:x_ptr+len(data_map[idx])] = np.ones(len(data_map[idx]))
        for file in data_map[idx]:
            im = imread(os.path.join(path_map[idx], file)).astype(np.float32)
            if im.shape != (h, w, c):
                im = im[:, :, :c]
            X[x_ptr] = im
            x_ptr += 1
    return X, Y

def prepare_dataset_for_retraining (dir_coke, dir_not_coke, default_transformations_coke, 
                                    default_transformations_not_coke, output_shape, create_augmentation=True):
    if len(dir_coke) == 0 and len(dir_not_coke) == 0:
        return None, None
    
    if len(default_transformations_coke) == 0:
        default_transformations_coke = ['flip_vertical_np', 'flip_horizontal_np', 'rotate_np', 'flip_rotate', 
        'perform_swirl_transformation', 'perform_random_affine_transform', 'add_multiplicative_noise', 'add_shot_noise', 
        'add_gaussian_noise', 'add_impulse_noise', 'add_glass_blur', 'add_gaussian_blur', 'random_image_eraser']
    if len(default_transformations_not_coke) == 0:
        default_transformations_not_coke = ['flip_rotate', 'random_image_eraser']
    
    all_data, all_paths, is_label = process_prepare_directories(dir_=dir_coke, transformations=default_transformations_coke, 
                                    label=True, create_augmentation=create_augmentation, data_map=[], paths_map=[], label_map=[])
    all_data, all_paths, is_label = process_prepare_directories(dir_=dir_not_coke, 
                                    transformations=default_transformations_not_coke, label=False, create_augmentation=create_augmentation, 
                                    data_map=all_data, paths_map=all_paths, label_map=is_label)
    
    X, Y = create_dataset(all_data, all_paths, is_label, output_shape)
    return X, Y

In [5]:
def retrain_model(dir_coke, dir_not_coke, dir_val_coke='', dir_val_not_coke='', default_transformations_coke=[], 
                  default_transformations_not_coke=[], augmentation_training=True, augmentation_validation=False, 
                  output_shape=160, model_params={}):
    # Creating the training dataset
    X_train, y_train = prepare_dataset_for_retraining(dir_coke, dir_not_coke,
                     default_transformations_coke, default_transformations_not_coke, output_shape, augmentation_training)
    
    if len(dir_val_coke) != 0 or len(dir_val_not_coke) !=0: # Use only when either of the path is specified
        # Creating the validation dataset
        X_val, y_val = prepare_dataset_for_retraining(dir_val_coke, dir_val_not_coke,
                 default_transformations_coke, default_transformations_not_coke, output_shape, augmentation_validation)
    y_train = to_categorical(y_train, num_classes=2)
    y_val = to_categorical(y_val, num_classes=2)
    return X_train, y_train, X_val, y_val
    
    model = CokeModel().get_model()
    
    # Default Model parameters
    default_max_epochs, default_split_size, default_batch_size = 10, 6144, 3072
    default_model_save_path = 'cokenet/model'
    
    max_epochs = model_params.get('max_epochs') or default_max_epochs
    split_size = model_params.get('split_size') or default_split_size
    batch_size = model_params.get('batch_size') or default_batch_size
    model_path = model_params.get('external_model_path') or ''
    
    idxs = np.arange(X_train.shape[0])
    for chunk in np.array_split(idxs, int(np.ceil(X_train.shape[0] / split_size))):
        X_current, y_current = X_train[chunk], y_train[chunk]
        if len(dir_val_coke) != 0 or len(dir_val_not_coke) !=0:
            model.fit(X_current, y_current, epochs=max_epochs, batch_size=batch_size, validation_data=(X_val, y_val))
        else:
            model.fit(X_current, y_current, epochs=max_epochs, batch_size=batch_size)
    
        if len(model_path) > 0:
            model.save(os.path.join(model_path, 'AlexNet.hdf5'))
        else:
            model.save(os.path.join(default_model_save_path, 'AlexNet.hdf5'))
        
    return model

In [ ]:
model_path111 = 'runnable_data/meraa'
os.path.join(model_path111, 'AlexNet.hdf5')

In [6]:
dir_coke = 'dataset/custom_trainable/coke'
dir_not_coke = 'dataset/custom_trainable/not_coke'
validation_coke = 'dataset/custom_trainable/val_coke'
validation_not_coke = 'dataset/custom_trainable/val_not_coke'
output_shape=160
coke, not_coke = True, True
default_transformations_coke = ['flip_vertical_np']
default_transformations_not_coke = ['flip_rotate']

X_train, y_train, X_val, y_val = retrain_model(dir_coke, dir_not_coke, validation_coke, validation_not_coke, 
                                              default_transformations_coke, default_transformations_not_coke)
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

In [12]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [7]:
model = CokeModel().get_model()
    
# Default Model parameters
default_max_epochs, default_split_size, default_batch_size = 10, 6144, 3072
default_model_save_path = 'cokenet/model'

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [ ]:
from tensorflow.python.keras.models import load_model
model = load_model('')

In [9]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
set_session(tf.Session(config=config))

In [19]:
from tensorflow.python.keras.backend import set_session
config = tf.ConfigProto()
sess = tf.Session(config=config)
graph = tf.get_default_graph()

# IMPORTANT: models have to be loaded AFTER SETTING THE SESSION for keras! 
# Otherwise, their weights will be unavailable in the threads after the session there has been set
set_session(sess)

In [20]:
model.compile(optimizer=RAdam(lr=0.01), loss='categorical_crossentropy')

In [22]:
global sess
global graph
with graph.as_default():
    set_session(sess)
    model.fit(X_train, y_train)

FailedPreconditionError: Error while reading resource variable conv2d_1/bias from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/conv2d_1/bias/class tensorflow::Var does not exist.
	 [[{{node conv2d_1/BiasAdd/ReadVariableOp}}]]
	 [[{{node loss_2/add_4}}]]

In [29]:
import tensorflow as tf
from keras import backend as k

In [30]:
session = tf.Session(graph=tf.Graph())
with session.graph.as_default():
    k.backend.set_session(session)
    model = CokeModel().get_model()

AttributeError: 'function' object has no attribute 'set_session'

In [46]:
gpus = tf.compat.v1.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpus[0], True)

tf.compat.v1.config.experimental.set_memory_growth(physical_devices[0], True) 

AttributeError: module 'tensorflow._api.v1.compat.v1.experimental' has no attribute 'list_physical_devices'

In [39]:
import keras 
tf.reset_default_graph() 
with tf.Session(config=config) as sess:
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.9)
    optimizer = RAdam(lr=0.01)
    init = tf.global_variables_initializer()
    sess.run(init)
    keras.backend.set_session(sess) 
    model = CokeModel().get_model()
    
    sess.run(tf.variables_initializer(optimizer.variables()))
    
    model.fit(X_train, y_train)

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d/Conv2D}}]]
	 [[{{node GroupCrossDeviceControlEdges_0/training_2/RAdam/RAdam/Const}}]]

In [19]:
model_params = {}
max_epochs = model_params.get('max_epochs') or 3
split_size = model_params.get('split_size') or default_split_size
batch_size = model_params.get('batch_size') or default_batch_size
model_path = model_params.get('external_model_path') or ''

In [29]:
import keras
import tensorflow as tf
keras.backend.get_session().run(tf.global_variables_initializer())

In [53]:
import keras
import tensorflow.keras.backend as K
import tensorflow as tf
cfg = tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(tf.Session(config=cfg))

In [50]:
sess = tf.Session()

In [54]:
idxs = np.arange(X_train.shape[0])
for chunk in np.array_split(idxs, int(np.ceil(X_train.shape[0] / split_size))):
    X_current, y_current = X_train[chunk], y_train[chunk]
    model.fit(X_current, y_current, epochs=max_epochs, batch_size=batch_size, validation_data=(X_val, y_val))

Train on 210 samples, validate on 23 samples
Epoch 1/3


FailedPreconditionError: Error while reading resource variable training_8/RAdam/iter from Container: localhost. This could mean that the variable was uninitialized. Not found: Container localhost does not exist. (Could not find resource: localhost/training_8/RAdam/iter)
	 [[{{node training_10/RAdam/RAdam/update_batch_normalization_2_2/beta/ReadVariableOp}}]]

In [60]:
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model

#tf_config = some_custom_config
sess = tf.Session()
graph = tf.get_default_graph()

# IMPORTANT: models have to be loaded AFTER SETTING THE SESSION for keras! 
# Otherwise, their weights will be unavailable in the threads after the session there has been set
set_session(sess)

In [76]:
model.get_losses_for()

TypeError: get_losses_for() missing 1 required positional argument: 'inputs'

In [77]:
from rectified_adam import RAdam
#model = CokeModel().get_model()
model.compile(optimizer=RAdam(lr=0.01), loss='categorical_crossentropy')

In [94]:
#activation, curr_epochs, optimizer = 'relu', max_epochs, RAdam(lr=0.01) 
#model = model_extractor(activation, X_train[0].shape)
model = CokeModel().get_model()
model.compile(loss="categorical_crossentropy", optimizer=RAdam(lr=0.01) , metrics=["accuracy"])

In [95]:
model.fit(X_train, y_train)

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_10/Conv2D}}]]
	 [[{{node loss_10/dense_3_loss/broadcast_weights/assert_broadcastable/is_valid_shape/has_valid_nonscalar_shape/has_invalid_dims/concat}}]]

In [ ]:
ctivation, curr_epochs, optimizer = 'relu', MAX_EPOCHS, RAdam(lr=0.01) 
model = model_extractor(activation, x_train[0].shape)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [85]:
X_train[0].shape

(160, 160, 3)

In [89]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, MaxPool2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD
from keras import regularizers
from keras.constraints import unit_norm

def model_extractor(activation_func, input_shape, weight_decay=1e-4):
    # Creating an AlexNet Classifier
    model = Sequential()
    
    constant_stride = (2, 2)
    constant_kernel = (5, 5)

    #Instantiating Layer 1
    model.add(Conv2D(48, kernel_size=constant_kernel, input_shape=input_shape, strides=constant_stride, activation=activation_func, padding='valid', 
                    kernel_constraint=unit_norm(), kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=constant_stride))

    # #Instantiating Layer 2
    model.add(Conv2D(96, kernel_size=constant_kernel, strides=constant_stride, activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=constant_stride))
    model.add(BatchNormalization())
    # #Instantiating Layer 3
    model.add(Conv2D(192, kernel_size=constant_kernel, strides=constant_stride, activation=activation_func, padding='same',
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(BatchNormalization())

    # #Instantiating Layer 4
    model.add(Conv2D(192, kernel_size=constant_kernel, strides=constant_stride, activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=constant_stride))
    model.add(BatchNormalization())

    # #Instantiating Layer 5
    model.add(Conv2D(256, kernel_size=constant_kernel, strides=constant_stride, activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(BatchNormalization())
    model.add(Flatten())

    #Instantiating Layer 6
    model.add(Dense(512, activation=activation_func)) 
    
    #Instantiating Layer 7
    model.add(Dense(256, activation=activation_func)) 

    #Instantiating Layer 8
    model.add(Dense(128, activation=activation_func)) 
    
    # #Instantiating Layer 9
    model.add(Dense(2, activation='softmax'))
    
    return model

In [ ]:
model = CokeModel().get_model()

In [10]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
max_epochs = 3
split_size = 6144
batch_size = int(split_size/2)

idxs = np.arange(X_train.shape[0])

for chunk in np.array_split(idxs, int(np.ceil(X_train.shape[0] / split_size))):
    X_current, y_current = X_train[chunk], y_train[chunk]
    model.fit(X_current, y_current, epochs=max_epochs, batch_size=batch_size, validation_data=(X_val, y_val))
    #model.save("runnable_data/model/{}2.hdf5".format(architecture))

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import tensorflow as tf
tf.config.list_logical_devices()

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
batch_size

In [ ]:
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
import sys
!conda install tensorflow-gpu

In [ ]:
def retrain_model(dir_coke, dir_not_coke, dir_val_coke='', dir_val_not_coke='', default_transformations_coke=[], 
                  default_transformations_not_coke=[], output_shape=160):
    # Setting up the variables
    X, Y = prepare_dataset_for_retraining(dir_coke, dir_not_coke, dir_val_coke,
                                         default_transformations_coke, default_transformations_not_coke, output_shape)

In [ ]:
dir_coke = 'dataset/custom_trainable/coke'
dir_not_coke = 'dataset/custom_trainable/not_coke'
validation_coke = 'dataset/custom_trainable/val_coke'
validation_not_coke = 'dataset/custom_trainable/val_not_coke'
output_shape=160
coke, not_coke = True, True
default_transformations_coke = ['flip_vertical_np']
default_transformations_not_coke = ['flip_rotate']

X, Y = prepare_dataset_for_retraining(dir_coke=dir_coke, dir_not_coke=dir_not_coke, dir_val_coke='',
                                      dir_val_not_coke='', coke=coke, not_coke=not_coke, validation=False,
                                      default_transformations_coke, default_transformations_not_coke, output_shape)

In [ ]:
X.shape, Y.shape

In [ ]:
model = CokeModel().get_model()

In [ ]:
model.we